In [1]:
from vampyr import vampyr3d as vp
from orbital4c import orbital as orb
import numpy as np
from scipy.linalg import eig, inv
import importlib
importlib.reload(orb)
from scipy.special import legendre, laguerre, erf, gamma
from scipy.special import gamma
from scipy.constants import hbar


c = 137   # NOT A GOOD WAY. MUST BE FIXED!!!
alpha = 1/c
k = -1
l = 0
n = 1
m = 0.5
Z = 2

mra = vp.MultiResolutionAnalysis(box=[-20,20], order=7)
prec = 1.0e-4
origin = [0.1, 0.2, 0.3]  # origin moved to avoid placing the nuclar charge on a node

print("init test orbital")
testorb1 = orb.orbital4c("X",mra)
testorb2 = orb.orbital4c("Y",mra)

a_coeff = 3.0
b_coeff = np.sqrt(a_coeff/np.pi)**3
gauss = vp.GaussFunc(b_coeff, a_coeff, origin)
gauss_tree = vp.FunctionTree(mra)
vp.advanced.build_grid(out=gauss_tree, inp=gauss)
vp.advanced.project(prec=prec, out=gauss_tree, inp=gauss)
gauss_tree.normalize()

testorb1.init_large_components(Lar = gauss_tree)
testorb1.init_small_components(prec/10)
testorb1.rescale(2)
testorb1.normalize()
testorb1.gradient()
testorb1.density(prec)

testorb2.init_large_components(Lbr = gauss_tree)
testorb2.init_small_components(prec/10)
testorb2.rescale(2)
testorb2.normalize()
testorb2.gradient()
testorb2.density(prec)

init test orbital

*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64


*FunctionTree: nn
  genNodes: 0
  square norm: 0
  root scale: -4
  order: 7
  nodes: 8
  endNodes: 8
  nodes per scale: 
    scale=-4  nodes=8
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0
  root scale: -4
  order: 7
  nodes: 8
  e

In [2]:
importlib.reload(orb)
spinor_1 = orb.orbital4c("helium_1",mra)
spinor_1.init_large_components(Lar = gauss_tree)
spinor_2 = orb.orbital4c("helium_2",mra)
spinor_2.init_large_components(Lbr = gauss_tree)

spinor_1.init_small_components(prec/10)
spinor_2.init_small_components(prec/10)
spinor_1.normalize()
spinor_2.normalize()

print(gauss_tree([0.5,0.5,0.5]))
print(spinor_1["Lar"]([0.5,0.5,0.5]))
print(spinor_2["Lai"]([0.5,0.5,0.5]))


*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0.933162
  root scale: -4
  order: 7
  nodes: 264
  endNodes: 232
  nodes per scale: 
    scale=-4  nodes=8
    scale=-3  nodes=64
    scale=-2  nodes=64
    scale=-1  nodes=64
    scale=0  nodes=64


*FunctionTree: nn
  genNodes: 0
  square norm: 0
  root scale: -4
  order: 7
  nodes: 8
  endNodes: 8
  nodes per scale: 
    scale=-4  nodes=8
 
*FunctionTree: nn
  genNodes: 0
  square norm: 0
  root scale: -4
  order: 7
  nodes: 8
  endNodes: 8
  nodes

In [3]:
def u(r):
    u = erf(r)/r + (1/(3*np.sqrt(np.pi)))*(np.exp(-(r**2)) + 16*np.exp(-4*r**2))
    #erf(r) is an error function that is supposed to stop the potential well from going to inf.
    #if i remember correctly
    return u

def V(x):
    r = np.sqrt(x[0]**2 + x[1]**2 + x[2]**2)
#    c = 0.0435
    c = 0.000435 # ten times tighter nuclear potential
    f_bar = u(r/c)/c
    return f_bar

Peps = vp.ScalingProjector(mra,prec)
f = lambda x: V([x[0]-origin[0],x[1]-origin[1],x[2]-origin[2]])

print(f([0.5,0.5,0.5]))
V_tree = Z*Peps(f)

1.8569533817705184


In [ ]:
#importlib.reload(orb)
#orbital_error_1 = 1
#while orbital_error_1 > prec:
#for i in range(1):    
#    hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
#    v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
#    add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
#    energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
#    print('Energy_1',energy_1,imag_1)
#    tmp_1 = orb.apply_helmholtz(v_psi_1, energy_1, c, prec)
#    new_orbital_1 = orb.apply_dirac_hamiltonian(tmp_1, prec, energy_1)
#    new_orbital_1.normalize()
#    delta_psi_1 = orb.add_orbitals(1.0, new_orbital_1, -1.0, spinor_1, prec)
#    orbital_error_1, imag_1 = orb.scalar_product(delta_psi_1, delta_psi_1)
#    print('Error_1',orbital_error_1, imag_1)
#    spinor_1 = new_orbital_1

#hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
#v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
#add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
#energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
#print('Energy_1',energy_1)

In [ ]:
#orbital_error_2 = 1
#while orbital_error_2 > prec:
#    hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)
#    v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)
#    add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)
#    energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)
#    print('Energy_2',energy_2,imag_2)
#    tmp_2 = orb.apply_helmholtz(v_psi_2, energy_2, c, prec)
#    new_orbital_2 = orb.apply_dirac_hamiltonian(tmp_2, prec, energy_2)
#    new_orbital_2.normalize()
#    delta_psi_2 = orb.add_orbitals(1.0, new_orbital_2, -1.0, spinor_2, prec)
#    orbital_error_2, imag_2 = orb.scalar_product(delta_psi_2, delta_psi_2)
#    print('Error_2',orbital_error_2, imag_2)
#    spinor_2 = new_orbital_2


#hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)
#v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)
#add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)
#energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)
#print('Energy_2',energy_2)

In [8]:
# Working on Helium with direct (J) = exchange (K)

importlib.reload(orb)
orbital_error = 1
while orbital_error > prec:

    #Definiton of Dirac Hamiltonian for alpha component   
    hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
    #Definiton of Dirac Hamiltonian for beta component 
    hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)


    #Definition of alpha density
    n_alpha = spinor_1.density(prec)
    #Definition of beta density
    n_beta = spinor_2.density(prec)


    #Definition of Poisson operator 
    Pua = vp.PoissonOperator(mra,prec=0.000001)
    #Defintion of J_alpha
    Puatree_alpha = Pua(n_alpha)*(4*np.pi)
    #Definiton of J_beta
    Puatree_beta = Pua(n_beta)*(4*np.pi)
    #Definition of Energy Hartree for alpha component
    E_H_alpha = vp.dot(n_alpha,Puatree_alpha)
    #Definition of Energy Hartree for beta component
    E_H_beta = vp.dot(n_beta,Puatree_beta)
    
    print("E_H_alpha",E_H_alpha)
    print("E_H_beta",E_H_beta)


    #Applying nuclear potential to spin orbital 
    v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
    v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)
    

    #Definition of full 4c hamitoninan 
    add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
    add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)
    

    #Orbital Energy calculation
    energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
    energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)
    energy_1 = energy_1 + E_H_alpha 
    energy_2 = energy_2 + E_H_beta 
    print('Energy_1',energy_1,imag_1)
    print('Energy_2',energy_2,imag_2)
    
    #Total Energy with J = K approximation 
    E_tot_J = energy_1 + energy_2 - E_H_alpha
    print("E_total(Coulomb) approximiation", E_tot_J)

    
    F12_spin = energy_12, imag_12 = orb.scalar_product(spinor_1, add_psi_2)
    F21_spin = energy_21, imag_21 = orb.scalar_product(spinor_2, add_psi_1)
    print("F12", F12_spin)
    print("F21", F21_spin)

    #Calculation of necessary potential contributions to Hellmotz 
    T1_spinor_1 = orb.apply_potential(V_tree, spinor_1, prec)
    T2_spinor_1 = orb.apply_potential(Puatree_alpha, spinor_1, prec)
    T3_spinor_1 = spinor_2.__rmul__(F12_spin)

    add_T1_T2_spinor_1 = orb.add_orbitals(1.0, T1_spinor_1, 1.0, T2_spinor_1, prec)
    add_T12_T3_spinor_1 = orb.add_orbitals(1.0, add_T1_T2_spinor_1, -1.0, T3_spinor_1, prec)

    T1_spinor_2 = orb.apply_potential(V_tree, spinor_2, prec)
    T2_spinor_2 = orb.apply_potential(Puatree_beta, spinor_2, prec)
    T3_spinor_2 = spinor_1.__rmul__(F21_spin)

    add_T1_T2_spinor_2 = orb.add_orbitals(1.0, T1_spinor_2, 1.0, T2_spinor_2, prec)
    add_T12_T3_spinor_2 = orb.add_orbitals(1.0, add_T1_T2_spinor_2, -1.0, T3_spinor_2, prec)

    #Calculation of Helmotz
    tmp_1 = orb.apply_helmholtz(add_T12_T3_spinor_1, energy_1, c, prec)
    tmp_2 = orb.apply_helmholtz(add_T12_T3_spinor_2, energy_2, c, prec)

    new_orbital_1 = orb.apply_dirac_hamiltonian(tmp_1, prec, energy_1)
    new_orbital_1.normalize()
    new_orbital_2 = orb.apply_dirac_hamiltonian(tmp_2, prec, energy_2)
    new_orbital_2.normalize()

    delta_psi_1 = orb.add_orbitals(1.0, new_orbital_1, -1.0, spinor_1, prec)
    orbital_error, imag_1 = orb.scalar_product(delta_psi_1, delta_psi_1)
    print('Error_1',orbital_error, imag_1)
    spinor_1 = new_orbital_1

    delta_psi_2 = orb.add_orbitals(1.0, new_orbital_2, -1.0, spinor_2, prec)
    orbital_error, imag_2 = orb.scalar_product(delta_psi_2, delta_psi_2)
    print('Error_2',orbital_error, imag_2)
    spinor_2 = new_orbital_2


#Definiton of Dirac Hamiltonian for alpha component   
hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
#Definiton of Dirac Hamiltonian for beta component 
hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)


#Definition of alpha density
n_alpha = spinor_1.density(prec)
#Definition of beta density
n_beta = spinor_2.density(prec)


#Definition of Poisson operator 
Pua = vp.PoissonOperator(mra,prec=0.000001)
#Defintion of J_alpha
Puatree_alpha = Pua(n_alpha)*(4*np.pi)
#Definiton of J_beta
Puatree_beta = Pua(n_beta)*(4*np.pi)
#Definition of Energy Hartree for alpha component
E_H_alpha = vp.dot(n_alpha,Puatree_alpha)
#Definition of Energy Hartree for beta component
E_H_beta = vp.dot(n_beta,Puatree_beta)
    
print("E_H_alpha",E_H_alpha)
print("E_H_beta",E_H_beta)


#Applying nuclear potential to spin orbital 
v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)
    

#Definition of full 4c hamitoninan 
add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)
    

#Orbital Energy calculation
energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)
energy_1 = energy_1 + E_H_alpha 
energy_2 = energy_2 + E_H_beta 
print('Energy_1',energy_1,imag_1)
print('Energy_2',energy_2,imag_2)
    
#Total Energy with J = K approximation 
E_tot_J = energy_1 + energy_2 - E_H_alpha
print("E_total(Coulomb) approximiation", E_tot_J)

E_H_alpha 0.06390536877694045
E_H_beta 0.06390536877929832
Energy_1 18768.8955951266 2.100820257027445e-10
Energy_2 18768.89559512661 -2.07580915042476e-10
E_total(Coulomb) approximiation 37537.72728488443
F12 (2791.8149165311775, 11960.100593482353)
F21 (2791.814916531179, -11960.10059348235)
mu 0.4569564159050367
mu 0.45695641588071284
Error_1 3.172627430048395 0.0
Error_2 3.1726274300519863 0.0
E_H_alpha 0.06485483589942047
E_H_beta 0.06485483589755081
Energy_1 18768.896080187533 -5.037164707086673e-10
Energy_2 18768.896080187533 4.997374313477532e-10
E_total(Coulomb) approximiation 37537.72730553916
F12 (-7798.417701073908, 9487.551305748666)
F21 (-7798.417701073908, -9487.551305748668)
mu 0.4558936822906448
mu 0.4558936822906448
Error_1 2.253253249324507 0.0
Error_2 2.253253249324161 0.0
E_H_alpha 0.06555406555372842
E_H_beta 0.06555406555527563
Energy_1 18768.89489815742 1.0987062593818646e-09
Energy_2 18768.894898157414 -1.1008663095154157e-09
E_total(Coulomb) approximiation 375

KeyboardInterrupt: 

In [7]:
# Working on Helium with direct and exchange
importlib.reload(orb)
orbital_error = 1
while orbital_error > prec:

    #Definiton of Dirac Hamiltonian for alpha component   
    hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
    #Definiton of Dirac Hamiltonian for beta component 
    hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)


    #Definition of alpha density
    n_alpha = spinor_1.density(prec)
    #Definition of beta density
    n_beta = spinor_2.density(prec)


    #Definition of Poisson operator 
    Pua = vp.PoissonOperator(mra,prec=0.000001)
 
    #Defintion of J_alpha
    Puatree_alpha = Pua(n_alpha)*(4*np.pi)
    Puatree_beta = Pua(n_beta)*(4*np.pi)

    #Definition of Energy Hartree for alpha component
    E_H_alpha = vp.dot(n_alpha,Puatree_beta)
    #Definition of Energy Hartree for beta component
    E_H_beta = vp.dot(n_beta,Puatree_alpha)
    
    print("E_H_alpha",E_H_alpha)
    print("E_H_beta",E_H_beta)


    #Definition of pseudo density made by differnt spin orbitals
    n_12 = spinor_2.exchange(spinor_1,prec)
    n_21 = spinor_1.exchange(spinor_2,prec)
    
    
    #Defintion of K
    K1_term = Pua(n_21)*(4*np.pi)
    K2_term = Pua(n_12)*(4*np.pi)

    #Defintion of K * spinorb 
    K1_term_spinor1 = orb.apply_potential(K1_term, spinor_1, prec)
    K2_term_spinor2 = orb.apply_potential(K2_term, spinor_2, prec)


    #Definition of Exchange energy term for each spin orbital
    K1_energy, imag_K1 = orb.scalar_product(spinor_2, K1_term_spinor1)
    #K1_energy = vp.dot(n_12,K1_term)

    K2_energy, imag_K2 = orb.scalar_product(spinor_1, K2_term_spinor2) 
    #K2_energy = vp.dot(n_21,K2_term)

    print("K1_energy",K1_energy)
    print("K2_energy",K2_energy)


    #Applying nuclear potential to spin orbital 
    v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
    v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)
    

    #Definition of full 4c hamitoninan 
    add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
    add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)
    

    #Orbital Energy calculation
    energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
    energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)
    energy_1 = energy_1 + (2*E_H_alpha) - K1_energy
    energy_2 = energy_2 + (2*E_H_beta) - K2_energy
    print('Energy_1',energy_1,imag_1)
    print('Energy_2',energy_2,imag_2)
    
    #Total Energy with direct and K 
    E_tot_JK = energy_1 + energy_2 - K1_energy
    print("E_total(direct & K) approximiation", E_tot_JK)

    
    F12_spin = energy_12, imag_12 = orb.scalar_product(spinor_1, add_psi_2)
    F21_spin = energy_21, imag_21 = orb.scalar_product(spinor_2, add_psi_1)
    print("F12", F12_spin)
    print("F21", F21_spin)

    #Calculation of necessary potential contributions to Hellmotz 
    T1_spinor_1 = orb.apply_potential(V_tree, spinor_1, prec)
    T2_spinor_1 = orb.apply_potential(Puatree_alpha, spinor_1, prec)
    T3_spinor_1 = orb.apply_potential(K1_term, spinor_1, prec)
    T4_spinor_1 = spinor_2.__rmul__(F12_spin)

    add_T1_T2_spinor_1 = orb.add_orbitals(1.0, T1_spinor_1, 2.0, T2_spinor_1, prec)
    add_T12_T3_spinor_1 = orb.add_orbitals(1.0, add_T1_T2_spinor_1, -1.0, T3_spinor_1, prec)
    add_T123_T4_spinor_1 = orb.add_orbitals(1.0, add_T12_T3_spinor_1, -1.0, T4_spinor_1, prec)

    T1_spinor_2 = orb.apply_potential(V_tree, spinor_2, prec)
    T2_spinor_2 = orb.apply_potential(Puatree_beta, spinor_2, prec)
    T3_spinor_2 = orb.apply_potential(K2_term, spinor_2, prec)
    T4_spinor_2 = spinor_1.__rmul__(F21_spin)


    add_T1_T2_spinor_2 = orb.add_orbitals(1.0, T1_spinor_2, 2.0, T2_spinor_2, prec)
    add_T12_T3_spinor_2 = orb.add_orbitals(1.0, add_T1_T2_spinor_2, -1.0, T3_spinor_2, prec)
    add_T123_T4_spinor_2 = orb.add_orbitals(1.0, add_T12_T3_spinor_2, -1.0, T4_spinor_2, prec)


    #Calculation of Helmotz
    tmp_1 = orb.apply_helmholtz(add_T123_T4_spinor_1, energy_1, c, prec)
    tmp_2 = orb.apply_helmholtz(add_T123_T4_spinor_2, energy_2, c, prec)

    new_orbital_1 = orb.apply_dirac_hamiltonian(tmp_1, prec, energy_1)
    new_orbital_1.normalize()
    new_orbital_2 = orb.apply_dirac_hamiltonian(tmp_2, prec, energy_2)
    new_orbital_2.normalize()

    # Calculate the overlap matrix
 #   S_ij = orb.scalar_product(new_orbital_1,new_orbital_2)
 #   sigma, U = eig(S_ij)
    
 #   Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()

    # Calculate the new orthogonalized orbitals
 #   new_orbital_1 = Sm5 @ new_orbital_1
 #   new_orbital_2 = Sm5 @ new_orbital_2 

    delta_psi_1 = orb.add_orbitals(1.0, new_orbital_1, -1.0, spinor_1, prec)
    orbital_error, imag_1 = orb.scalar_product(delta_psi_1, delta_psi_1)
    print('Error_1',orbital_error, imag_1)
    spinor_1 = new_orbital_1

    delta_psi_2 = orb.add_orbitals(1.0, new_orbital_2, -1.0, spinor_2, prec)
    orbital_error, imag_2 = orb.scalar_product(delta_psi_2, delta_psi_2)
    print('Error_2',orbital_error, imag_2)
    spinor_2 = new_orbital_2



#Definiton of Dirac Hamiltonian for alpha component   
hd_psi_1 = orb.apply_dirac_hamiltonian(spinor_1, prec)
#Definiton of Dirac Hamiltonian for beta component 
hd_psi_2 = orb.apply_dirac_hamiltonian(spinor_2, prec)


#Applying nuclear potential to spin orbital 
v_psi_1 = orb.apply_potential(V_tree, spinor_1, prec)
v_psi_2 = orb.apply_potential(V_tree, spinor_2, prec)

#Definition of full 4c hamitoninan 
add_psi_1 = orb.add_orbitals(1.0, hd_psi_1, 1.0, v_psi_1, prec)
add_psi_2 = orb.add_orbitals(1.0, hd_psi_2, 1.0, v_psi_2, prec)

#Orbital Energy calculation
energy_1, imag_1 = orb.scalar_product(spinor_1, add_psi_1)
energy_2, imag_2 = orb.scalar_product(spinor_2, add_psi_2)


#Definition of alpha density
n_alpha = spinor_1.density(prec)
#Definition of beta density
n_beta = spinor_2.density(prec)


#Definition of Poisson operator 
Pua = vp.PoissonOperator(mra,prec=0.000001)

#Defintion of J_alpha
Puatree_alpha = Pua(n_alpha)*(4*np.pi)
Puatree_beta = Pua(n_beta)*(4*np.pi)

#Definition of Energy Hartree for alpha component
E_H_alpha = vp.dot(n_alpha,Puatree_alpha)
#Definition of Energy Hartree for beta component
E_H_beta = vp.dot(n_beta,Puatree_beta)

    
print("E_H_alpha",E_H_alpha)
print("E_H_beta",E_H_beta)

   
#Definition of pseudo density made by differnt spin orbitals
n_12 = spinor_2.exchange(spinor_1,prec)
n_21 = spinor_1.exchange(spinor_2,prec)
    
    
#Defintion of K
K1_term = Pua(n_21)*(4*np.pi)
K2_term = Pua(n_12)*(4*np.pi)

#Defintion of K * spinorb 
K1_term_spinor1 = orb.apply_potential(K1_term, spinor_1, prec)
K2_term_spinor2 = orb.apply_potential(K2_term, spinor_2, prec)


#Definition of Exchange energy term for each spin orbital
K1_energy, imag_K1 = orb.scalar_product(spinor_2, K1_term_spinor1)
#K1_energy = vp.dot(n_12,K1_term)   
K2_energy, imag_K2 = orb.scalar_product(spinor_1, K2_term_spinor2)
#K2_energy = vp.dot(n_21,K2_term)

print("K1_energy",K1_energy)
print("K2_energy",K2_energy)


#Total Energy with direct and K 
E_tot_JK = energy_1 + energy_2 + E_H_alpha + E_H_beta - K1_energy
print("E_total(direct & K) approximiation", E_tot_JK)

E_H_alpha 0.11785447742628086
E_H_beta 0.1178544774262809
K1_energy -0.044192353937390676
K2_energy -0.044192353937390676
Energy_1 18768.95831990648 -6.981998828024067e-09
Energy_2 18768.95831990648 6.981203876998217e-09
E_total(direct & K) approximiation 37537.9608321669
F12 (-11407.933353594348, 5213.576399371078)
F21 (-11407.933353594353, -5213.5763993710825)
mu 0.28872148254123864
mu 0.28872148254123864
Error_1 1.1449400812984485 0.0
Error_2 1.1449400812965504 0.0
E_H_alpha 0.1002532642270037
E_H_beta 0.10025326422700369
K1_energy -0.003503966240643876
K2_energy -0.00350396624064387
Energy_1 18768.92873389423 3.0039412915949196e-09
Energy_2 18768.928733894227 -3.0098530342479455e-09
E_total(direct & K) approximiation 37537.8609717547
F12 (-3490.1546976637464, -11918.326752347512)
F21 (-3490.1546976637483, 11918.326752347512)
mu 0.37753402620173315
mu 0.3775340262143506
Error_1 3.106938825801714 0.0
Error_2 3.106938825801369 0.0
E_H_alpha 0.09224296257659004
E_H_beta 0.0922429625765

KeyboardInterrupt: 

In [ ]:
importlib.reload(orb)
exact_orbital = orb.orbital4c("exact_1s",mra)
orb.init_1s_orbital(exact_orbital,k,Z,n,alpha,origin,prec)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
fig, axs = plt.subplots(2,2,figsize=(10, 10))
#axs[0,0].set_box_aspect(1)
#axs[0,1].set_box_aspect(1)
#axs[1,0].set_box_aspect(1)
#axs[1,1].set_box_aspect(1)
x_plt = np.linspace(-1, 1, 1000)
orbital_plt = {}
exact_plt = {}
out_plt = {}
orb_plt = {}
add_psi_plt = {}
exact_plt['Lar']   = [exact_orbital['Lar'](   [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
exact_plt['Sai']   = [exact_orbital['Sai'](   [  origin[0],   origin[1], x+origin[2]]) for x in x_plt]
exact_plt['Sbr']   = [exact_orbital['Sbr'](   [  origin[0], x+origin[1],   origin[2]]) for x in x_plt]
exact_plt['Sbi']   = [exact_orbital['Sbi'](   [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
exact_plt['Lar']   = [exact_orbital['Lar'](   [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
exact_plt['Sai']   = [exact_orbital['Sai'](   [  origin[0],   origin[1], x+origin[2]]) for x in x_plt]
exact_plt['Sbr']   = [exact_orbital['Sbr'](   [  origin[0], x+origin[1],   origin[2]]) for x in x_plt]
exact_plt['Sbi']   = [exact_orbital['Sbi'](   [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
orb_plt['Lar']     = [spinor_1['Lar'](         [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
orb_plt['Sai']     = [spinor_1['Sai'](         [  origin[0],   origin[1], x+origin[2]]) for x in x_plt]
orb_plt['Sbr']     = [spinor_1['Sbr'](         [  origin[0], x+origin[1],   origin[2]]) for x in x_plt]
orb_plt['Sbi']     = [spinor_1['Sbi'](         [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
orb_plt['Lar']     = [spinor_2['Lar'](         [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
orb_plt['Sai']     = [spinor_2['Sai'](         [  origin[0],   origin[1], x+origin[2]]) for x in x_plt]
orb_plt['Sbr']     = [spinor_2['Sbr'](         [  origin[0], x+origin[1],   origin[2]]) for x in x_plt]
orb_plt['Sbi']     = [spinor_2['Sbi'](         [x+origin[0],   origin[1],   origin[2]]) for x in x_plt]
#plt.plot(x_plt, [add_psi_plt['Lar'][i]/orbital_plt['Lar'][i] for i in range(1000)])
#axs[0,0].plot(x_plt, add_psi_plt['Lar'],label='add_psi')
#axs[0,0].plot(x_plt, orbital_plt['Lar'],label='gaussian')
axs[0,0].plot(x_plt, exact_plt['Lar'],label='exact')
#axs[0,0].plot(x_plt, out_plt['Lar'],label='out (1 iter)')
axs[0,0].plot(x_plt, orb_plt['Lar'],label='out (err 10^-4)')
axs[0,0].legend()
#axs[0,1].plot(x_plt, add_psi_plt['Sai'],label='add_psi')
#axs[0,1].plot(x_plt, orbital_plt['Sai'],label='gaussian')
axs[0,1].plot(x_plt, exact_plt['Sai'],label='exact')
#axs[0,1].plot(x_plt, out_plt['Sai'],label='out (1 iter)')
axs[0,1].plot(x_plt, orb_plt['Sai'],label='out (err 10^-4)')
axs[0,1].legend()
#axs[1,0].plot(x_plt, add_psi_plt['Sbr'],label='add_psi')
#axs[1,0].plot(x_plt, orbital_plt['Sbr'],label='gaussian')
axs[1,0].plot(x_plt, exact_plt['Sbr'],label='exact')
#axs[1,0].plot(x_plt, out_plt['Sbr'],label='out (1 iter)')
axs[1,0].plot(x_plt, orb_plt['Sbr'],label='out (err 10^-4)')
axs[1,0].legend()
#axs[1,1].plot(x_plt, add_psi_plt['Sbi'],label='add_psi')
#axs[1,1].plot(x_plt, orbital_plt['Sbi'],label='gaussian')
axs[1,1].plot(x_plt, exact_plt['Sbi'],label='exact')
#axs[1,1].plot(x_plt, out_plt['Sbi'],label='out (1 iter)')
axs[1,1].plot(x_plt, orb_plt['Sbi'],label='out (err 10^-4)')
axs[1,1].legend()